In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy.special import expit  # Sigmoid function for propensity score
from kernel_regression import KernelRegression

In [7]:
# Simulation parameters
n = 3000 # Number of samples
p = 3   # Number of covariates
treatment_effect = 5.0  # True treatment effect

# Generate covariates
np.random.seed(0)

In [8]:
true_ATE = treatment_effect

### Data generation

In [11]:
num_trial = 300

In [12]:
nsims = 1000
np.random.seed(123)
methods = ['dr', 'direct', 'ips']
srr = {'dr' : True, 'direct' : False, 'ips' : True}

In [906]:
import numpy as np
from scipy import optimize


class RKHS_RieszLearner:
    def __init__(self, method, model_name, link_name):
        if method == "LS":
            self.loss_func = self.ls_loss
        if method == "KL":
            self.loss_func = self.kl_loss
        if method == "EB":
            self.loss_func = self.eb_loss
        
        self.model_name = model_name
        self.link_name = link_name

    def _model_construction(self, param, X1, X0, treatment):
        if self.link_name == "Linear":
            if (self.model_name == "Common"):
                param1 = param[:int(len(param)/2)]
                param0 = param[int(len(param)/2):]
                fx1 = X1 @ param1
                fx0 = X0 @ param0
            else:
                fx1 = X1 @ param
                fx0 = X0 @ param
            alpha1 = fx1
            alpha0 = fx0
            alpha = alpha0
            alpha[treatment] = alpha1[treatment]
            
        if self.link_name == "Logistic":
            fx1 = X1 @ param
            ex1 = 1/(1 + np.exp(-fx1))
            fx0 = X0 @ param
            ex0 = 1/(1 + np.exp(-fx0))
            alpha = treatment / ex1 - (1 - treatment) / (1 - ex0)

        return alpha
        
    def ls_loss(self, param, X1, X0, treatment, regularizer):
        treatment0 = treatment*0
        treatment1 = treatment0 + 1
        alpha1 = self._model_construction(param, X1, X0, treatment1)
        alpha0 = self._model_construction(param, X1, X0, treatment0)
        loss = - 2*(alpha1 - alpha0) + treatment*alpha1**2 + (1 - treatment)*alpha0**2
        loss = np.mean(loss) + regularizer * np.sum(param**2)
        return loss
    
    def kl_loss(self, param, X1, X0, treatment, regularizer):
        treatment0 = treatment*0
        treatment1 = treatment0 + 1
        alpha1 = self._model_construction(param, X1, X0, treatment1)
        alpha0 = self._model_construction(param, X1, X0, treatment0)
        loss = - np.log(alpha1) - np.log(-alpha0) + treatment*alpha1 - (1 - treatment)*alpha0
        loss = np.mean(loss) + regularizer * np.sum(param**2)
        return loss
    
    def eb_loss(self, param, X1, X0, treatment, regularizer):
        treatment0 = treatment*0
        treatment1 = treatment0 + 1
        alpha1 = self._model_construction(param, X1, X0, treatment1)
        alpha0 = self._model_construction(param, X1, X0, treatment0)
        loss = - (1 - treatment)*np.log(alpha1 - 1) - treatment*np.log(- alpha0 - 1) + treatment*alpha1 - (1 - treatment)*alpha0
        loss = np.mean(loss) + regularizer * np.sum(param**2)
        print(loss)
        return loss
        
    def optimize(self, covariate, treatment, x_test):
        result = self.minimize(covariate, treatment, 0.01)
        self.x_test = x_test
        
    def obj_func_gen(self, X1, X0, treatment, regularizer):                
        obj_func = lambda param: self.loss_func(param, X1, X0, treatment, regularizer)
        return obj_func
    
    def fit(self, X1, X0, treatment, regularizer):        
        obj_func = self.obj_func_gen(X1, X0, treatment, regularizer)
        
        if (self.model_name == "Common") & (self.link_name == "Linear"):
            init_param = np.zeros(X1.shape[1]*2)
        else:
            init_param = np.zeros(X1.shape[1])

        self.result = optimize.minimize(obj_func, init_param, method="BFGS")
        self.params = self.result.x
                
        return self.params
    
    def test(self, x, b, t, quant=True, pi=False):
        theta = 0
        f = np.dot(x, b)
        if quant is True:
            temp = np.copy(f)
            temp = np.sort(temp)
            theta = temp[np.int(np.floor(len(x)*(1-pi)))]
        pred = np.zeros(len(x))
        pred[f > theta] = 1
        acc = np.mean(pred == t)
        return acc

    def dist(self, X, X1, X0, treatment=None, num_basis=False):
        (d,n) = X.shape
        
        if num_basis is False:
            num_basis = 1000

        idx = np.random.permutation(n)[0:num_basis]
        C = X[:, idx]

        # calculate the squared distances
        X1C_dist = CalcDistanceSquared(X1, C)
        X0C_dist = CalcDistanceSquared(X0, C)
        DC_dist = CalcDistanceSquared(treatment, C)
        CC_dist = CalcDistanceSquared(C, C)
        return X1C_dist, X0C_dist, DC_dist, CC_dist, n, num_basis


    def kernel_cv(self, covariate_train, treatment, covariate_test, folds=5, num_basis=False, sigma_list=None, lda_list=None):
        if self.model_name == "Separate":
            treatment0 = treatment*0
            treatment1 = treatment0 + 1
            X_train1 = np.concatenate([np.array([treatment1]).T, covariate_train], axis=1)
            X_train0 = np.concatenate([np.array([treatment0]).T, covariate_train], axis=1)
            X_train = np.concatenate([np.array([treatment]).T, covariate_train], axis=1)
        elif self.model_name == "Common":
            X_train1 = covariate_train
            X_train0 = covariate_train
            X_train = covariate_train
                    
        if self.model_name == "Separate":
            X_test = np.concatenate([np.array([treatment]).T, covariate_test], axis=1)
        elif self.model_name == "Common":
            X_test = covariate_test
            
        X_train, X_train1, X_train0, X_test = X_train.T, X_train1.T, X_train0.T, X_test.T
        X1C_dist, X0C_dist, DC_dist, CC_dist, n, num_basis = self.dist(X_train, X_train1, X_train0, X_test, num_basis)
        # setup the cross validation
        cv_fold = np.arange(folds) # normal range behaves strange with == sign
        cv_split0 = np.floor(np.arange(n)*folds/n)
        cv_index = cv_split0[np.random.permutation(n)]
        # set the sigma list and lambda list
        if sigma_list==None:
            sigma_list = np.array([0.01, 0.05, 0.1, 0.5, 1])
        if lda_list==None:
            lda_list = np.array([0.01, 0.05, 0.1, 0.5, 1])
        score_cv = np.zeros((len(sigma_list), len(lda_list)))
        
        for sigma_idx, sigma in enumerate(sigma_list):
            # pre-sum to speed up calculation
            h1_cv = []
            h0_cv = []
            d_cv = []
            for k in cv_fold:
                h1_cv.append(np.exp(-X1C_dist[:, cv_index==k]/(2*sigma**2)))
                h0_cv.append(np.exp(-X0C_dist[:, cv_index==k]/(2*sigma**2)))
                d_cv.append(treatment[cv_index==k])

            for k in range(folds):
                #print(h0_cv[0])
                # calculate the h vectors for training and test
                count = 0
                for j in range(folds):
                    if j == k:
                        h1te = h1_cv[j].T
                        h0te = h0_cv[j].T
                        dte = d_cv[j]
                    else:
                        if count == 0:
                            h1tr = h1_cv[j].T
                            h0tr = h0_cv[j].T
                            dtr = d_cv[j]
                            count += 1
                        else:
                            h1tr = np.append(h1tr, h1_cv[j].T, axis=0)
                            h0tr = np.append(h0tr, h0_cv[j].T, axis=0)
                            dtr = np.append(dtr, d_cv[j], axis=0)

                one = np.ones((len(h1tr),1))
                h1tr = np.concatenate([h1tr, one], axis=1)
                h0tr = np.concatenate([h0tr, one], axis=1)
                one = np.ones((len(h1te),1))
                h1te = np.concatenate([h1te, one], axis=1)
                h0te = np.concatenate([h0te, one], axis=1)
                for lda_idx, lda in enumerate(lda_list):
                    res_param = self.fit(h1tr, h0tr, dtr, lda)
                    # calculate the solution and cross-validation value
                    obj_func = self.obj_func_gen(h1te, h0te, dte, 0)
                    score = obj_func(res_param)       
                    score_cv[sigma_idx, lda_idx] = score_cv[sigma_idx, lda_idx] + score


        # get the minimum
        (sigma_idx_chosen, lda_idx_chosen) = np.unravel_index(np.argmin(score_cv), score_cv.shape)
        sigma_chosen = sigma_list[sigma_idx_chosen]
        lda_chosen = lda_list[lda_idx_chosen]

        x1_train = np.exp(-X1C_dist/(2*sigma_chosen**2)).T
        x0_train = np.exp(-X0C_dist/(2*sigma_chosen**2)).T
        x_test = np.exp(-DC_dist/(2*sigma_chosen**2)).T

        one = np.ones((len(x1_train),1))
        X1_train = np.concatenate([x1_train, one], axis=1)
        X0_train = np.concatenate([x0_train, one], axis=1)
        one = np.ones((len(x_test),1))
        X_test = np.concatenate([x_test, one], axis=1)

        print(34)
        print(sigma_idx_chosen, lda_idx_chosen)
        print(score_cv)
        return X1_train, X0_train, X_test, lda_chosen



def CalcDistanceSquared(X, C):
    '''
    Calculates the squared distance between X and C.
    XC_dist2 = CalcDistSquared(X, C)
    [XC_dist2]_{ij} = ||X[:, j] - C[:, i]||2
    :param X: dxn: First set of vectors
    :param C: d:nc Second set of vectors
    :return: XC_dist2: The squared distance nc x n
    '''

    Xsum = np.sum(X**2, axis=0).T
    Csum = np.sum(C**2, axis=0)
    XC_dist = Xsum[np.newaxis, :] + Csum[:, np.newaxis] - 2*np.dot(C.T, X)
    return XC_dist


In [907]:
X_train

array([[3.18565118e-11, 6.79439056e-05, 3.22634022e-13, ...,
        1.84059842e-09, 2.47619531e-06, 1.00000000e+00],
       [2.75622293e-02, 3.22251834e-02, 6.15494899e-05, ...,
        1.06047907e-05, 4.25503599e-04, 1.00000000e+00],
       [1.22726048e-03, 1.61491224e-02, 8.83784599e-08, ...,
        2.75158156e-06, 9.83835031e-06, 1.00000000e+00],
       ...,
       [3.22669988e-04, 6.14002595e-04, 1.15061884e-04, ...,
        7.44212513e-08, 7.19893974e-04, 1.00000000e+00],
       [1.25119513e-01, 2.13852673e-02, 7.40176963e-04, ...,
        6.76752796e-04, 2.08688479e-04, 1.00000000e+00],
       [6.53007955e-01, 3.73671254e-03, 2.38549688e-03, ...,
        1.46523109e-05, 4.72847434e-05, 1.00000000e+00]], shape=(6000, 51))

In [925]:
model = RKHS_RieszLearner("LS", "Common", "Logistic")
model = RKHS_RieszLearner("LS", "Separate", "Linear")
model = RKHS_RieszLearner("KL", "Common", "Logistic")
#model = RKHS_RieszLearner("EB", "Common", "Logistic")


In [926]:
X1_train, X0_train, X_test, lda_chosen = model.kernel_cv(covariate, treatment, covariate, folds=2, num_basis=50)

34
4 0
[[1.10824566 1.11012325 1.11587363 1.15998682 1.18387747]
 [1.10820052 1.11011403 1.11586944 1.15998624 1.1838772 ]
 [1.10802471 1.11008038 1.11585443 1.1599847  1.18387656]
 [0.90062739 1.03097409 1.07358876 1.15180056 1.17944551]
 [0.6347628  0.84556489 0.9424132  1.0989529  1.14040871]]


In [927]:
X1_train

array([[0.02373279, 0.3231449 , 0.01873902, ..., 0.00223222, 0.0300584 ,
        1.        ],
       [0.38445838, 0.09808698, 0.69215863, ..., 0.14695711, 0.39044814,
        1.        ],
       [0.07263832, 0.28381672, 0.18880815, ..., 0.0109396 , 0.64433763,
        1.        ],
       ...,
       [0.97687847, 0.04870433, 0.66426542, ..., 0.57197381, 0.05521792,
        1.        ],
       [0.08937845, 0.43614616, 0.28891683, ..., 0.03396159, 1.        ,
        1.        ],
       [0.21443161, 0.09499593, 0.55045058, ..., 0.10753226, 0.52794689,
        1.        ]], shape=(3000, 51))

In [928]:
X0_train

array([[0.02373279, 0.3231449 , 0.01873902, ..., 0.00223222, 0.0300584 ,
        1.        ],
       [0.38445838, 0.09808698, 0.69215863, ..., 0.14695711, 0.39044814,
        1.        ],
       [0.07263832, 0.28381672, 0.18880815, ..., 0.0109396 , 0.64433763,
        1.        ],
       ...,
       [0.97687847, 0.04870433, 0.66426542, ..., 0.57197381, 0.05521792,
        1.        ],
       [0.08937845, 0.43614616, 0.28891683, ..., 0.03396159, 1.        ,
        1.        ],
       [0.21443161, 0.09499593, 0.55045058, ..., 0.10753226, 0.52794689,
        1.        ]], shape=(3000, 51))

In [929]:
param = model.fit(X1_train, X0_train, treatment, lda_chosen)

In [930]:
param

array([ 0.22290776, -0.26819854,  0.18263007, -0.0572777 ,  0.19897182,
       -0.21363251, -0.00721066,  0.45429154, -0.06635056,  0.00962513,
       -0.47264793,  0.29921256,  1.07548308, -0.16059467, -0.14323398,
        0.07830309,  0.06265214,  0.25293698,  0.418337  , -0.2275189 ,
       -0.01130893, -0.09943221, -0.12464728, -0.00139151,  0.04561794,
        0.07747579,  0.56854383,  0.173371  , -0.38057247, -0.13678693,
        0.1789083 , -0.55442239,  0.67900037,  0.28122544, -0.13518248,
       -0.10261297,  0.18990896,  0.07894066,  0.24863382,  0.31490512,
        0.27941754, -0.31034535, -0.14270491, -0.71864442,  0.3095028 ,
       -0.10728542,  0.32880536,  0.36380871, -0.21887687, -0.34264033,
       -1.28300225])

In [931]:
a = X_test @ param

In [932]:
X_test.shape

(3000, 51)

In [933]:
e = 1/(1 + np.exp(-a))

In [934]:
e

array([0.18425408, 0.16558271, 0.12904567, ..., 0.4526736 , 0.18269181,
       0.13731756], shape=(3000,))

In [935]:
a

array([-1.48778727, -1.61726279, -1.90942314, ..., -0.18987399,
       -1.49821564, -1.83775043], shape=(3000,))

In [936]:
propensity_scores

array([0.0345672 , 0.16820832, 0.02368561, ..., 0.47667238, 0.16818766,
       0.12640262], shape=(3000,))

In [937]:
np.mean(treatment / e * outcome)

np.float64(7.903922609236567)

In [938]:
np.mean((1 - treatment) / (1 - e) * outcome)

np.float64(3.18809604334634)

In [939]:
alpha = treatment / e - (1 - treatment) / (1 - e)

In [923]:
np.mean(alpha * outcome)

np.float64(4.5535276031829435)

In [940]:
np.mean(alpha * outcome)

np.float64(4.715826565890227)

In [401]:
treatment_effect

5.0

In [409]:
result_list = []

for tr in range(num_trial):
    result_list_temp = []
    
    # Generate covariates
    covariate = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    covariate_temp = np.concatenate([covariate, covariate**2, np.array([covariate[:, 0]*covariate[:, 1], 
                                                                        covariate[:, 1]*covariate[:, 2], 
                                                                        covariate[:, 0]*covariate[:, 2]]).T], axis=1)
    propensity_coef = np.random.normal(0, 0.5, covariate_temp.shape[1])
    propensity_scores = expit(covariate_temp @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    treatment = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    beta1 = np.random.normal(0, 1, p)
    gamma1 = np.random.normal(0, 1, p)
    fx1 = covariate @ beta1
    gx1 = covariate**2 @ gamma1
    outcome = (fx1)**2 + 1/(1 + np.exp(-gx1)) + 1.1 + treatment_effect * treatment + np.random.normal(0, 1, n)
    
    
    covariate_treatment = X[T == 1]
    X_control = X[T == 0]
    
    y_scaler = StandardScaler(with_mean=True).fit(np.array([Y]).T)
    y = y_scaler.transform(np.array([Y]).T)
    XT = np.c_[T, X]
    
    X_train, X_test, y_train, y_test = train_test_split(XT, y, test_size = 0.2)

    torch.cuda.empty_cache()
    learner = Learner(X_train.shape[1], n_hidden, drop_prob, 0, interaction_only=True)
    agmm = RieszNet(learner, moment_fn)
    # Fast training
    agmm.fit(X_train, y_train, Xval=X_test, yval=y_test,
             earlystop_rounds=2, earlystop_delta=earlystop_delta,
             learner_lr=1e-4, learner_l2=learner_l2, learner_l1=learner_l1,
             n_epochs=100, bs=bs, target_reg=target_reg,
             riesz_weight=riesz_weight, optimizer='adam',
             model_dir=str(Path.home()), device=device)
    # Fine tune
    agmm.fit(X_train, y_train, Xval=X_test, yval=y_test,
             earlystop_rounds=earlystop_rounds, earlystop_delta=earlystop_delta,
             learner_lr=learner_lr, learner_l2=learner_l2, learner_l1=learner_l1,
             n_epochs=600, bs=bs, target_reg=target_reg,
             riesz_weight=riesz_weight, optimizer='adam', warm_start=True,
             model_dir=str(Path.home()), device=device)
    
    params = tuple(x * y_scaler.scale_[0] for method in methods
                   for x in agmm.predict_avg_moment(XT, y,  model='final', method = method, srr = srr[method])) + (true_ATE, )
                        
    result_list2.append(params)
    
    Y = y.T[0]
    
    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
    XT = np.c_[T, X]
    
    outcome_model = MLPRegressor(random_state=1, max_iter=600)
    outcome_model.fit(XT, Y)

    T_treatment = T*0 + 1
    T_control = T*0
    XT_treatment = np.c_[T_treatment, X]
    XT_control = np.c_[T_control, X]

    est_treatment_outcome = outcome_model.predict(XT_treatment)
    est_control_outcome = outcome_model.predict(XT_control)
    
    for method in ["DBCLS", "DBCUKL", "CBPS", "Logit"]:
        prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0., loss=method)
        prop_model.fit(X, T)
        est_prop_score = prop_model.predict_proba(X)[:, 1]
        est_prop_score_dbc = est_prop_score

        #treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
        #control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

        IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

        # Evaluate performance
        IPW_bias = IPW_est - true_ATE

        result_list_temp.append(IPW_est)

        DM_est = np.mean(est_treatment_outcome - est_control_outcome)

        # Evaluate performance
        DM_bias = DM_est - true_ATE
        
        result_list_temp.append(DM_est)

        DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

        # Evaluate performance
        DR_bias = DR_est - true_ATE

        result_list_temp.append(DR_est)
    
    ##### Linear models
    
    # Fit a linear model to estimate the treatment effect
    model = LinearRegression()
    model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
    estimated_treatment_effect = model.coef_[-1]

    # Evaluate performance
    true_ATE = treatment_effect
    bias = estimated_treatment_effect - true_ATE
    mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

    result_list_temp.append(estimated_treatment_effect)
    
    #### CBPS
    
    # Enable automatic conversion of Pandas DataFrame to R DataFrame
    pandas2ri.activate()

    # Simulate data in Python

    # Create a pandas DataFrame
    column_names = [f'X{i+1}' for i in range(p)]
    df = pd.DataFrame(X, columns=column_names)
    df['T'] = T
    df['Y'] = Y


    # Convert pandas DataFrame to R DataFrame
    r_df = pandas2ri.py2rpy(df)

    ro.r.assign("p", p)

    # Load the CBPS package in R and fit the model for ATE estimation
    ro.r('''
        library(CBPS)
        estimate_cbps_ate <- function(df) {
            formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))

            # CBPSの適用 (ATEの推定、ATT=0)
            model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")

            # 推定された傾向スコアの取得
            df$propensity_score <- fitted(model)

            # IPW (Inverse Probability Weighting) を適用
            df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))

            # 重み付き回帰によるATEの推定
            result <- lm(Y ~ T, data = df, weights = df$weight)

            return(df$propensity_score)
        }
    ''')

    # R関数を呼び出してATEと傾向スコアを取得
    est_prop_score = ro.r['estimate_cbps_ate'](r_df)
    
    est_prop_score_cbps = est_prop_score
    
    #print(er)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE
    
    result_list_temp.append(DR_est)
    
    result_list_temp = np.array(result_list_temp)*y_scaler.scale_[0]
    
    result_list.append(result_list_temp)
    
    res = tuple(np.array(x) for x in zip(*result_list2))
    truth = res[-1:]
    res_dict = {}

    res_list_temp = []
    for it, method in enumerate(methods):
        point, lb, ub = res[it * 3: (it + 1)*3]
        res_list_temp.append(point)
        
    result_list_final = np.concatenate([np.array(result_list), np.array(res_list_temp).T], axis=1)
    
    print(np.round(np.sqrt(np.mean((result_list_final - true_ATE)**2, axis=0)), 3))

KeyboardInterrupt: 

In [40]:
Y

array([23.71877552,  7.80900173,  5.1638035 , ...,  6.21743204,
        3.04346554,  7.66068435], shape=(3000,))

In [36]:
# Generate covariates
X = np.random.normal(0, 1, (n, p))

# Define a propensity score model
X_temp = np.concatenate([X, X**2, np.array([X[:, 0]*X[:, 1], X[:, 1]*X[:, 2], X[:, 0]*X[:, 2]]).T], axis=1)
propensity_coef = np.random.normal(0, 0.5, X_temp.shape[1])
propensity_scores = expit(X_temp @ propensity_coef)  # Calculate propensity scores

# Generate treatment assignment based on propensity scores
T = np.random.binomial(1, propensity_scores)

# Generate outcome with treatment effect
beta1 = np.random.normal(0, 1, p)
gamma1 = np.random.normal(0, 1, p)
fx1 = X @ beta1
gx1 = X**2 @ gamma1
Y1 = (fx1)**2 + 3/(1 + np.exp(-gx1)) + 1.1 + np.random.normal(0, 1, n)

beta0 = np.random.normal(0, 1, p)
gamma0 = np.random.normal(0, 1, p)

fx0 = X @ beta0
Y0 = (fx0 + 1)**2 + 1.1 + np.random.normal(0, 1, n)

In [37]:
adgustmenttreatment_effect / np.mean(Y1 - Y0) 

np.float64(-1.9172335774802356)